In [28]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import pickle
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
matplotlib 2.1.2
numpy 1.19.1
pandas 0.22.0
sklearn 0.19.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [29]:
# 从本地保存数据文件加载数据
with open(file='../chapter_2/data/california_housing.pkl', mode='rb') as f:
    housing=pickle.load(f)
# from sklearn.datasets import fetch_california_housing
# housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [30]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [16]:
# metric使用
# 对应元素差的平方求和，然后再求平均（单个数组中元素的个数）
metric = keras.metrics.MeanSquaredError()
print(metric([5., 4.], [2., 2.]))

tf.Tensor(6.5, shape=(), dtype=float32)


In [17]:
print(metric([5.], [2.])) 
# keras.metrics下面的指标是累积的，在当前batch上的结果会和之前的batch做平均。而keras.losses下面的不会。
# 上一次的是6.5，这一次的是9，平均之后就是7.75
print(metric([0.], [1.]))
print(metric.result())

tf.Tensor(7.75, shape=(), dtype=float32)
tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor(5.5, shape=(), dtype=float32)


In [18]:
# 具有累加功能，不想累加要reset_states
metric.reset_states()
metric([1.], [3.])
print(metric.result())

tf.Tensor(4.0, shape=(), dtype=float32)


In [37]:
# 1. batch 遍历训练集 metric
#    1.1 自动求导
# 2. epoch结束 验证集 metric

epochs = 100
batch_size = 32
steps_per_epoch = len(x_train_scaled) // batch_size
optimizer = keras.optimizers.SGD()
metric = keras.metrics.MeanSquaredError()

def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]

model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])

for epoch in range(epochs):
    
    metric.reset_states()
    for step in range(steps_per_epoch):
        x_batch, y_batch = random_batch(x_train_scaled, y_train,
                                        batch_size)
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
#             y_pred = tf.squeeze(y_pred, 1)
            loss = keras.losses.mean_squared_error(y_batch, y_pred)
            metric(y_batch, y_pred)
        grads = tape.gradient(loss, model.variables)
        grads_and_vars = zip(grads, model.variables)
        optimizer.apply_gradients(grads_and_vars)
        print("\rEpoch", epoch, " train mse:",
              metric.result().numpy(), end="")
    y_valid_pred = model(x_valid_scaled)
    y_valid_pred = tf.squeeze(y_valid_pred, 1)
    valid_loss = keras.losses.mean_squared_error(y_valid_pred, y_valid)
    print("\t", "valid mse: ", valid_loss.numpy())

Epoch 0  train mse: 1.3330629	 valid mse:  1.345685336597863
Epoch 1  train mse: 1.2864465	 valid mse:  1.3862601861174022
Epoch 2  train mse: 1.2705623	 valid mse:  1.3499298901068846
Epoch 3  train mse: 1.3395284	 valid mse:  1.3604443347362967
Epoch 4  train mse: nan	 valid mse:  nan
Epoch 5  train mse: nan	 valid mse:  nan
Epoch 6  train mse: nan	 valid mse:  nan
Epoch 7  train mse: nan	 valid mse:  nan
Epoch 8  train mse: nan	 valid mse:  nan
Epoch 9  train mse: nan	 valid mse:  nan
Epoch 10  train mse: nan	 valid mse:  nan
Epoch 11  train mse: nan	 valid mse:  nan
Epoch 12  train mse: nan	 valid mse:  nan
Epoch 13  train mse: nan	 valid mse:  nan
Epoch 14  train mse: nan	 valid mse:  nan
Epoch 15  train mse: nan	 valid mse:  nan
Epoch 16  train mse: nan	 valid mse:  nan
Epoch 17  train mse: nan	 valid mse:  nan
Epoch 18  train mse: nan	 valid mse:  nan
Epoch 19  train mse: nan	 valid mse:  nan
Epoch 20  train mse: nan	 valid mse:  nan
Epoch 21  train mse: nan	 valid mse:  nan
Epo